# 16장. 코루틴

이 장에서는 다음 내용들을 설명한다.

* 제너레이터를 코루틴으로 만드는 방법
* 코루틴으로 작동하는 제너레이터의 동작과 상태
* 제너레이터 객체의 `close()`와 `throw()` 메서드를 통해 호출자가 코루틴을 제어하는 방법
* 종료할 때 코루틴이 값을 반환하는 방법
* 새로운 `yield from` 구문의 사용법과 의미
* 사용 예: 시뮬레이션의 동시 활동을 관리하기 위한 코루틴

## 16.1 코루틴은 제너레이터에서 어떻게 진화했는가?

...

## 16.2 코루틴으로 사용되는 제너레이터의 기본 동작

다음 예제는 코루틴의 동작을 보여준다.

In [7]:
def simple_coroutine(): # 코루틴은 본체 안에 yield문을 가진 일종의 제너레이터 함수로 정의된다
    print('-> coroutine started')
    x = yield # yield문을 표현식에 사용한다.
    print('-> coroutine received:', x)

In [8]:
my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x7f09e82d9930>

In [9]:
next(my_coro) # 제너레이터를 yield문까지 실행함으로써 데이터를 전송할 수 있는 상태를 만든다.

-> coroutine started


In [10]:
my_coro.send(42) # send() 메서드를 호출해서 본체 안의 yield문의 값을 42로 만든다.
# 이제 코루틴이 실행되어서 다음 yield문이 나오거나 종료될 때까지 실행된다.

-> coroutine received: 42


StopIteration: 

제어 흐름이 코루틴 본체의 끝에 도달하므로 일반적인 제너레이터와 마찬가지로 `StopIteration` 예외를 발생시킨다.

코루틴은 다음 네 가지 상태를 가지며, `inspect.getgeneratorstate()` 함수를 이용해 현재 상태를 알 수 있다.

* `GEN_CREATED` : 실행을 위해 대기하고 있는 상태
* `GEN_RUNNING` : 현재 인터프리터가 실행하고 있는 상태
* `GEN_SUSPENDED` : 현재 yield 문에서 대기하고 있는 상태
* `GEN_CLOSED` : 실행이 완료된 상태

`send()` 메서드에 전달한 인수가 대기하고 있는 yield 표현식의 값이 되므로, 코루틴이 대기 상태에 있을 때는 `my_coro.send(42)`와 같은 형태로만 호출할 수 있다. 그러나 코루틴이 아직 기동되지 않은 `GEN_CREATED` 상태인 경우에는 `send()` 메서드를 호출할 수 없다. 그래서 코루틴을 처음 활성화하기 위해 `next(my_coro)`를 호출한다.

코루틴 객체를 생성한 직후 `None`이 아닌 값을 전달하려고 하면 다음과 같이 오류가 발생한다.

In [11]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

처음 `next(my_coro)`를 호출할 때 코루틴을 기동한다. 즉 처음 yield문까지 실행을 진행한다. 

yield문이 두 번 이상 나오는 예제를 보자.

In [12]:
def simple_coro2(a):
    print('-> Started: a=', a)
    b = yield a
    print('-> Received: b=', b)
    c = yield a + b
    print('-> Received: c=', c)

In [21]:
my_coro2 = simple_coro2(14)

In [22]:
from inspect import getgeneratorstate
getgeneratorstate(my_coro2) # 아직 실행안됨

'GEN_CREATED'

In [23]:
next(my_coro2) # 첫번째 yield문까지 진행하면서 메세지를 출력하고 a의 값을 생성한 후 b에 값이 할당될 때까지 기다린다.

-> Started: a= 14


14

In [24]:
getgeneratorstate(my_coro2) # 코루틴이 yield문에서 대기하고 있는 상태

'GEN_SUSPENDED'

In [25]:
my_coro2.send(28) # 중단된 코루틴에 28을 보내면 yield문의 값은 28로 평가되고 이 값이 b에 할당된다. 그후 a+b의 값(42)가 생성된다.

-> Received: b= 28


42

In [26]:
my_coro2.send(99) # yield는 99로 평가되고 코루틴은 종료된다.

-> Received: c= 99


StopIteration: 

In [27]:
getgeneratorstate(my_coro2) # 종료된 상태

'GEN_CLOSED'

코루틴의 할당문에서는 실제 값을 할당하기 전에 = 오른쪽 코드를 실행한다. 즉 `b = yield a`에서는 나중에 호출자가 값을 보낸 후에야 변수 b가 설정된다. 

## 16.3 예제: 이동 평균을 계산하는 코루틴

약간 더 복잡한 코루틴 예제를 살펴보자.

In [31]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True: 
        # 이 코루틴은 호출자가 close() 메서드를 호출하거나 
        # 객체에 대한 참조가 모두 사려저서 가비지 컬렉트되어야 종료됨
        term = yield average
        total += term
        count += 1
        average = total/count

코루틴을 사용하면 total과 count를 지역 변수로 사용할 수 있다.

In [36]:
coro_avg = averager()
next(coro_avg) # average의 초깃값 None 반환

In [33]:
coro_avg.send(10)

10.0

In [34]:
coro_avg.send(30)

20.0

In [35]:
coro_avg.send(5)

15.0

## 16.4 코루틴을 기동하기 위한 데커레이터

코루틴은 반드시 `next(my_coro)`를 호출해야하는데, 이를 편리하게 사용할 수 있도록 기동하는 데커레이터가 종종 사용된다. 대표적으로 `@coroutine`이 사용된다.

* 일반적인 데커레이터는 키워드 인수를 지원하지 않으며, 데커레이트된 함수의 `__name__`과 `__doc__` 속성을 가린다. 이를 functools.wraps() 데커레이터를 이용해서 해결할 수 있다.

In [37]:
# coroutil.py

from functools import wraps

def coroutine(func):
    """데커레이터: 'func'를 기동해서 첫번째 yield까지 진행한다."""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen) # 제너레이터를 기동한다.
        return gen # 제너레이터를 반환한다.
    return primer

In [38]:
from coroutil import coroutine

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

이제 돌려보자

In [40]:
coro_avg = averager()
from inspect import getgeneratorstate
getgeneratorstate(coro_avg) # 코루틴이 값을 받을 준비가 되어 있다.

'GEN_SUSPENDED'

In [41]:
coro_avg.send(10) # coro_avg 객체에 바로 값을 전송할 수 있다.

10.0

In [42]:
coro_avg.send(30)

20.0

In [43]:
coro_avg.send(5)

15.0

## 16.5 코루틴 종료와 예외 처리

코루틴 안에서 발생한 예외를 처리하지 않으면, `next()`나 `send()`로 코루틴을 호출한 호출자에 예외가 전파된다.

In [44]:
from coroaverager1 import averager
coro_avg = averager()
coro_avg.send(40)

40.0

In [46]:
coro_avg.send(50)

45.0

In [47]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [49]:
coro_avg.send(60) # 코루틴 안에서 예외를 처리하지 않으므로 코루틴이 종료됨

StopIteration: 

위 예제처럼 코루틴에 구분 표시를 전송해서 코루틴을 종료할 수 있다. 

제너레이터 객체는 호출자가 코루틴에 명시적으로 예외를 전달할 수 있게 해주는 `throw()`와 `close()` 메서드를 제공한다.

* generator.throw(exc_type[, exc_value[, traceback]])
  : 제너레이터가 중단한 곳의 yield 표현식에 예외를 전달한다. 제너레이터가 예외를 처리하면 제어 흐름이 다음 yield문까지 진행하고, 생성된 값은 `generator.throw()` 호출 값이 된다. 제너레이터가 예외를 처리하지 않으면 호출자까지 예외가 전파된다.

* generator.close()
  : 제너레이터가 실행을 중단한 yield 표현식이 GeneratorExit 예외를 발생시키게 만든다. 제너레이터가 예외를 처리하지 않거나 StopIteration 예외를 발생시키면 아무러 ㄴ에러도 호출자에 전달되지 않는다.

In [50]:
# coro_exc_demo.py

class DemoException(Exception):
    """설명에 사용할 예외 유형"""

def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else: # 예외가 발생하지 않으면 받은 값을 출력한다.
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.') # 이 코드는 실행되지 않음

이제 예외가 처리되므로 `demo_exc_handling()` 코루틴은 계속 실행된다.

In [51]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [52]:
exc_coro.send(11)

-> coroutine received: 11


In [53]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [54]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

처리되지 않는 예외를 더니면 코루틴이 중단된다.

In [55]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [56]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

코루틴이 어떻게 종료되든 어떤 정리 코드를 실행해야 하는 경우에는 `try/finally` 블록 안에 해당 코드를 넣어야 한다.

In [57]:
# coro_finally_demo.py

class DemoException(Exception):
    """설명에 사용할 예외 유형"""

def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else: # 예외가 발생하지 않으면 받은 값을 출력한다.
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

## 16.6 코루틴에서 값 반환하기

코루틴은 값을 반환할 수도 있다.

In [58]:
# coroaverager2.py

from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break # 값을 반환하려면 코루틴이 정상적으로 종료되어야 한다.
            # 따라서 이 averager 버전에서는 루프를 빠져나오는 조건을 검사한다.
        total += term
        count += 1
        average = total/count
    return Result(count, average) # nametuple 반환

In [67]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10) # 값을 생성하지 않음

In [68]:
coro_avg.send(30)

In [69]:
coro_avg.send(6.5)

In [70]:
coro_avg.send(None) # None을 보내면 루프를 빠져나오고 코루틴이 결과를 반환하면서 종료
# 예외 객체의 value 속성에는 반환된 값이 들어있다.

StopIteration: Result(count=3, average=15.5)

코루틴이 반환한 값을 가져오려면 다음과 같이 하면 된다.

In [71]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10) # 값을 생성하지 않음

In [72]:
coro_avg.send(30)

In [73]:
coro_avg.send(6.5)

In [74]:
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value

In [75]:
result

Result(count=3, average=15.5)

## 16.7 yield from 사용하기

`yield from`은 완전히 새로운 언어 구성체이다. 제너레이터 `gen()`이 `yield from subgen()`을 호출하고, `subgen()`이 이어받아 값을 생성하고 `gen()`의 호출자에 반환한다.

14장에서 `yield from`을 for 루프 안의 yield에 대한 단축문으로 사용할 수 있다고 설명했다.

In [77]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i

In [78]:
list(gen())

['A', 'B', 1, 2]

위 코드는 다음과 같이 바꿀 수 있다.

In [79]:
def gen():
    yield from 'AB'
    yield from range(1,3)
    
list(gen())

['A', 'B', 1, 2]

앞에서 나왔던 예제는 다음과 같았다.

In [80]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [81]:
s = 'ABC'
t = tuple(range(3))
list(chain(s,t))

['A', 'B', 'C', 0, 1, 2]

`yield from x` 표현식이 x 객체에 대해 첫 번째로 하는 일은 `iter(x)`를 호출해서 x의 반복자를 가져오는 것이다. 그러나 `yield from`의 주요한 특징은 가장 바깥쪽 호출자와 가장 안쪽에 있는 하위 제너레이터 사이에 양방향 채널을 열어준다는 것이다.

`yield from`을 사용하기 위한 주요 용어는 다음과 같다.

* `대표 제너레이터`
  : `yield from <반복형>` 표현식을 담고 있는 제너레이터 함수

* `하위 제너레이터`
  : `yield from` 표현식 중 <반복형>에서 가져오는 제너레이터
  
* `호출자`
  : 대표 제너레이터를 호출하는 코드

다음 스크립트는 가상의 중1 남학생과 여학생의 몸무게와 키를 담은 딕셔너리를 읽는다. 예를 들어 `boys;m` 키는 남학생 9명의 키에 매핑되고, `girls;kg` 키는 여학생 10명의 몸무게에 매핑된다.

In [91]:
# coroaverager3.py

from collections import namedtuple

Result = namedtuple('Result', 'count average')

# 하위 제너레이터
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield # main() 안의 클라이언트가 전송하는 값이 여기에 바인딩
        if term is None: # 종료 조건
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average) #반환값은 groper()의 yield from의 값이 된다

# 대표 제너레이터
def grouper(results, key):
    while True: # 반복때마다 하나의 averager() 객체 생성
        # grouper()가 값을 받을 때마다 이 값은 yield from에 의해 averager() 객체로 전달된다
        # grouper()는 클라이언트가 전송한 값들을 averager()가 소진할 때까지 여기에서 중단
        # averager() 객체가 실행을 완료하고 반환한 값은 results[key]에 바인딩
        results[key] = yield from averager()
        
# 호출자
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key) # 코루틴 생성
        next(group) # 코루틴 기동
        for value in values:
            group.send(value) # 값을 하나씩 grouper()에 전달
            # 이 값은 averager()의 term = yield 값이 된다
        group.send(None) # 이 부분이 중요!
        # None을 전달하면 현재 avergaer() 객체가 종료되고 grouper()가 실행을 재개
        # 여기가 없으면 스크립트는 아무것도 출력하지 않는다
    
    print(results)
    report(results)

# 실행 결과 보고서
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
            result.count, group, result.average, unit))
        
data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
}
        
if __name__ == '__main__':
    main(data)

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [90]:
!python3 coroaverager3.py

{'girls;kg': Result(count=10, average=42.040000000000006), 'girls;m': Result(count=10, average=1.4279999999999997), 'boys;kg': Result(count=9, average=40.422222222222224), 'boys;m': Result(count=9, average=1.3888888888888888)}
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


작동과정은 다음과 같다.

* 바깥쪽 for 루프를 반복할 때마다 group이라는 이름의 `grouper()` 객체를 새로 생성한다.
* `next(group)`을 호출해서 `grouper()` 대표 제너레이터를 기동시킨다. 그러면 하위 제너레이터 `averager()`를 호출한 후 `yield from`에서 대기한다.
* 내부 for 루프에서 `group.send(value)`를 호출해서 `averager()`에 데이터 전달
* 내부 for 루프가 끝났을 때 `grouper()` 객체는 여전히 `yield from`에 멈춰있으므로 `grouper()` 본체 안의 `results[key]`에 대한 할당은 아직 실행되지 않는다.
* 바깥쪽 for 루프에서 마지막으로 `group.send(None)`을 호출하지 않으면, 하위 제너레이터인 `averager()`의 실행이 종료되지 않으므로, 대표 제너레이턴 group이 다시 활성화되지 않고 결국 `results[key]`에 아무런 값도 할당되지 않는다.
* 바깥쪽 for 루프가 다시 반복되면 새로운 grouper() 객체가 생성되며 기존 객체는 가비지 컬렉트 된다.

모든 `yield from` 체인은 가작 바깥쪽 대표 제너레이터에 `next()`와 `send()`를 호출하는 클라이언트에 의해 주도된다. 

## 16.8 yield from의 의미

`yield from`의 동작은 다음과 같다.

* 하위 제너레이터가 생성하는 값은 모두 대표 제너레이터의 호출자에 바로 전달
* `send()`